# Gausian Elimination

Given a linear system of $n$ equations in $n$ unknowns, *Gausian Elimination* can deliver the solution.


Here is an example problem:

\begin{align}
x + y + z &= 9 \\
2x -3y + 4z &= 13 \\
3x + 4y + 5x &= 40
\end{align}

We will solve this system of equations by first creating an array containing both the coefficients and right-hand-side values. We will use **numpy** to set this up:

In [3]:
import numpy as np

In [8]:
n = 3
coeffs = np.zeros((n, n+1))

*Numpy* creates an $nxn+1$ array initialized with all zero values. Next, we load the data:

In [9]:
coeffs[0][0] = 1
coeffs[0][1] = 1
coeffs[0][2] = 1
coeffs[0][3] = 9
coeffs[1][0] = 2
coeffs[1][1] = -3
coeffs[1][2] = 4
coeffs[1][3] = 13
coeffs[2][0] = 3
coeffs[2][1] = 4
coeffs[2][2] = 5
coeffs[2][3] =40
coeffs

array([[ 1.,  1.,  1.,  9.],
       [ 2., -3.,  4., 13.],
       [ 3.,  4.,  5., 40.]])

The basic scheme we are going to use involves applying some basic operations on one row in this matrix, then combining that row with the row below. The goal in these operations is to drive the matrix of left-hand side values to something called *upper-diagonal form* where all values below the diagonal are zero.

This process is going to destroy the original matrix, so we will work on a copy of the matrix in this process.

The first operation we apply basically scaled the elements in the first row so that the number on the diagonal has a value of one. We do this by dividing each element in the row by the value on the diagonal. Of course, that diagonal must not be zero, so we need to check that.

In the next operation we are going to subtract this row from the row below

In [10]:
a = np.copy(coeffs)
for i in range(n):
    # check for zero diagonal values - stop the process if zero
    if a[i][i] == 0.0:
        print('Divide by zero detected!')
        break
    
    # scale the elements on the rows below this one.
    for j in range(i+1,n):
        # calculate the scale factor needed for the next row
        scale = a[j][i]/a[i][i]
        
        # subtract using scale on the elements on this row
        for k in range(n+1):
            a[j][k] = a[j][k] - scale * a[i][k]

This process should drive the array into *upper-diagonal* form:

In [67]:
a

array([[ 1. ,  1. ,  1. ,  9. ],
       [ 0. , -5. ,  2. , -5. ],
       [ 0. ,  0. ,  2.4, 12. ]])

Once n this form, the solution can be obtained by working from the bottom up. The last row provides the answer for the left most unknown directly. We substitute this into the row above, eliminating the unknown from below leaving another simple solution. 

We will build the solution in a new array:

In [12]:
x = np.zeros(n)

The correct value for the rightmost unknown is sitting in the **a** array, use that to solve for the next unknown to the left on the row above. 

In [14]:
x[n-1] = a[n-1][n]/a[n-1][n-1]

Now, we loop over the remaining rows, moving upward. On each row, we use the new *known* values to eliminate all unknowns to the right. Note that we only need to work on elements to the right of the diagonal, since those to the left are all zero.

In [15]:
for i in range(n-2,-1,-1):
    x[i] = a[i][n]
    
    for j in range(i+1,n):
        x[i] = x[i] - a[i][j]*x[j]
    x[i] = x[i]/a[i][i]

In [16]:
x

array([1., 3., 5.])

We can package all of this in a simple function that takes the coefficient array as a parameter, and returns the solution vector

In [28]:
import numpy as np

def gauss(coeffs):
    a = np.copy(coeffs)
    n = len(a)
    # drive to upper-diagonal form
    for i in range(n):
        if a[i][i] == 0.0:
            print('Divide by zero detected!')
            break
        for j in range(i+1,n):
            scale = a[j][i]/a[i][i]
            for k in range(n+1):
                a[j][k] = a[j][k] - scale * a[i][k]
    # backfill
    x = np.zeros(n)
    x[n-1] = a[n-1][n]/a[n-1][n-1]
    for i in range(n-2,-1,-1):
        x[i] = a[i][n]
        for j in range(i+1,n):
            x[i] = x[i] - a[i][j]*x[j]
        x[i] = x[i]/a[i][i]
    return x

In [29]:
sol = gauss(coeffs)

In [30]:
a

array([[ 1. ,  1. ,  1. ,  9. ],
       [ 0. , -5. ,  2. , -5. ],
       [ 0. ,  0. ,  2.4, 12. ]])

In [32]:
coeffs

array([[ 1.,  1.,  1.,  9.],
       [ 2., -3.,  4., 13.],
       [ 3.,  4.,  5., 40.]])

In [33]:
sol

array([1., 3., 5.])

In [34]:
%%writefile pygauss.py
def gauss(coeffs):
    a = np.copy(coeffs)
    n = len(a)
    # drive to upper-diagonal form
    for i in range(n):
        if a[i][i] == 0.0:
            print('Divide by zero detected!')
            break
        for j in range(i+1,n):
            scale = a[j][i]/a[i][i]
            for k in range(n+1):
                a[j][k] = a[j][k] - scale * a[i][k]
    # backfill
    x = np.zeros(n)
    x[n-1] = a[n-1][n]/a[n-1][n-1]
    for i in range(n-2,-1,-1):
        x[i] = a[i][n]
        for j in range(i+1,n):
            x[i] = x[i] - a[i][j]*x[j]
        x[i] = x[i]/a[i][i]
    return x

Writing pygauss.py
